### **1. Make classification and get it ready**

In [21]:
import sklearn

from sklearn.datasets import make_circles

# Make 1000 samples 
n_samples = 1010
X, y = make_circles(n_samples, noise=0.03, random_state=42)

In [6]:
len(X), len(y)

(1010, 1010)

In [22]:
print(f'First 5 samples of X: \n {X[:5]}')
print(f'First 5 samples of y:\n {y[:5]}')

First 5 samples of X: 
 [[ 0.04119107  0.769308  ]
 [-0.67138971 -0.31448208]
 [-0.49455616  0.63858398]
 [ 0.80606474  0.12846176]
 [ 0.78511667  0.23101545]]
First 5 samples of y:
 [1 1 1 1 1]


In [27]:
X.shape, y.shape, X.ndim, y.ndim

((1010, 2), (1010,), 2, 1)

In [33]:
X[:5, :]

array([[ 0.04119107,  0.769308  ],
       [-0.67138971, -0.31448208],
       [-0.49455616,  0.63858398],
       [ 0.80606474,  0.12846176],
       [ 0.78511667,  0.23101545]])

In [34]:
X[:5, : 1]

array([[ 0.04119107],
       [-0.67138971],
       [-0.49455616],
       [ 0.80606474],
       [ 0.78511667]])

In [35]:
X[:5, 1:]

array([[ 0.769308  ],
       [-0.31448208],
       [ 0.63858398],
       [ 0.12846176],
       [ 0.23101545]])

In [ ]:
# Make DataFrame of circle data
import pandas as pd
circles = pd.DataFrame({'X1' : X[] 
                        })